In [1]:
'''get list all files ending from a given folder  with Path module'''

from pathlib import Path
import pandas as pd

import numpy as np 
import csv



In [26]:


def get_all_files_from_folder(folder_path, file_extension):
    """get list all files ending from a given folder  with Path module"""
    folder_path = Path(folder_path)
    file_list = list(folder_path.glob(f'*.{file_extension}'))
    return file_list

def flatten_clipping_dict2csv_format(clipping_dict, file_properties_df,excerpt_duration=10):
    """flatten clipping array to csv format

    """
    existing_files = set([str(file) for file in file_properties_df.index])

    def get_file_row(file_path,existing_files):
        """get file row from file_properties_df"""
        if file_path not in existing_files:
            return None,None,None
        file_row=file_properties_df.loc[Path(file_path)]
        region  = file_row['region']
        location = file_row['locationId']
        timestamp = file_row['timestamp']
        return region, location, timestamp

    
    def sort_files_by_timestamp(clipping_dict,existing_files):
        """sort files by timestamp to make sure we have the correct order"""
        max_channel = 0
        clipping_files_per_location = []
        for file_path,clipping in clipping_dict.items():
            max_channel = max(max_channel,clipping.shape[1])
            region, location, timestamp = get_file_row(file_path,existing_files)
            if timestamp is None:
                continue
            clipping_files_per_location.append((file_path,region,location,timestamp,max_channel))
            # sort by timestamp
            clipping_files_per_location.sort(key=lambda x: x[3])
        return clipping_files_per_location, max_channel
    
    clipping_files_per_location, max_channel = sort_files_by_timestamp(clipping_dict,existing_files)

    csv_list = []
    for file_info in clipping_files_per_location:
        file_path,region,location,timestamp,max_channel = file_info
        clipping = clipping_dict[file_path]

        for clipping_values in clipping:
            # format floats with four digits after the decimal point
            clipping_values = [ f'{x:.4f}' for x in clipping_values]
            timestamp_str = timestamp.strftime('%Y-%m-%d_%H:%M:%S')
            csv_list.append((file_path.replace('/tank/data/nna/real/',''),timestamp_str,*clipping_values))
            timestamp += pd.Timedelta(seconds=excerpt_duration)
    return csv_list, max_channel


def save_csv_file(csv_list, max_channel, file_path):
    """save csv file with csv library"""
    with open(file_path, 'w') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['file_path','timestamp']+['channel_'+str(i) for i in range(max_channel)])
        csvwriter.writerows(csv_list)
    return True


In [22]:
OUTPUT_FOLDER = Path('/scratch/enis/data/nna/clipping_info/csv_exports/all-merged_2022-09-12')
INPUT_FOLDER = Path('/scratch/enis/data/nna/clipping_info/all-merged_2022-09-12')
file_extension = 'pkl'
clipping_files = get_all_files_from_folder(str(INPUT_FOLDER), file_extension)

file_database = '/scratch/enis/data/nna/database/allFields_dataV102.pkl'
file_properties_df = pd.read_pickle(file_database)


In [23]:
# OUTPUT_FOLDER

# NEEED TO SORT THE FILES


PosixPath('/scratch/enis/data/nna/clipping_info/csv_exports/all-merged_2022-09-12')

In [30]:

OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)

for clipping_file in clipping_files:
    clipping_info = np.load(str(clipping_file), allow_pickle=True)[()]
    csv_file_per_loc,max_channel = flatten_clipping_dict2csv_format(clipping_info, file_properties_df,excerpt_duration=10)
    file_path  = OUTPUT_FOLDER / f'{clipping_file.stem}.csv'
    save_csv_file(csv_file_per_loc, max_channel,str(file_path))


In [29]:
csv_file_per_loc[0]

('dalton/10/2019/S4A10350_20190318_122302.flac',
 '2019-03-18_12:23:02',
 '0.3806',
 '0.3211')